In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import cv2 as cv
from tqdm import tqdm
import os
import json

from lac.plotting import plot_path_3d
from lac.localization.mono_vo import MonoVisualOdometry

%load_ext autoreload
%autoreload 2

# Feature detection


In [ ]:
data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/data_collection_1")

i = 100
I1 = cv.imread(os.path.join(data_path, "front_left", f"{i}.png"), cv.IMREAD_GRAYSCALE)
I2 = cv.imread(os.path.join(data_path, "front_left", f"{i + 10}.png"), cv.IMREAD_GRAYSCALE)

# Plot images side by side
fig, ax = plt.subplots(1, 2, figsize=(15, 15))
ax[0].imshow(I1)
ax[1].imshow(I2)
plt.show()

ORB (left) vs FAST (right)


In [ ]:
fast_detector = cv.FastFeatureDetector_create(threshold=25, nonmaxSuppression=True)
orb_detector = cv.ORB_create(nfeatures=1000)
orb_kp = orb_detector.detect(I1)
orb_kp, orb_des = orb_detector.compute(I1, orb_kp)

fast_kp = fast_detector.detect(I1)

I1_orb = cv.drawKeypoints(I1, orb_kp, None, color=(0, 255, 0), flags=0)
I1_fast = cv.drawKeypoints(I1, fast_kp, None, color=(0, 255, 0), flags=0)
fig, ax = plt.subplots(1, 2, figsize=(15, 15))
ax[0].imshow(I1_orb)
ax[1].imshow(I1_fast)
plt.show()

In [ ]:
I2_fast = cv.drawKeypoints(I2, fast_detector.detect(I2), None, color=(0, 255, 0), flags=0)
fig, ax = plt.subplots(1, 2, figsize=(15, 15))
ax[0].imshow(I1_fast)
ax[1].imshow(I2_fast)
plt.show()

# Mono VO class


In [11]:
## Parameters ---------------------------------------------------------------

data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/data_collection_1")
img_path = f"{data_path}/front_left"
json_data = json.load(open(f"{data_path}/data_log.json"))
initial_pose = np.array(json_data["initial_pose"])  # TODO: initialize with initial pose

trajlen = 1000

# for KITTI
focal = 915.0
pp = (1280 / 2, 720 / 2)  # principal point

# Parameters for lucas kanade optical flow
lk_params = dict(
    winSize=(21, 21), criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 30, 0.01)
)

In [8]:
poses = []
positions = []
for frame in json_data["frames"]:
    poses.append(np.array(frame["pose"]))
    positions.append(np.array(frame["pose"])[:3, 3])
positions = np.array(positions)

In [4]:
vo = MonoVisualOdometry(img_path, poses, focal, pp, lk_params)
vo_traj = np.zeros((trajlen, 3))

for i in range(trajlen):
    vo.process_frame()
    vo_traj[i, :] = vo.get_mono_coordinates()

In [ ]:
fig = plot_path_3d(vo_traj[:, 0], vo_traj[:, 1], vo_traj[:, 2])
fig = plot_path_3d(positions[:trajlen, 0], positions[:trajlen, 1], positions[:trajlen, 2], fig=fig)
fig.show()